# Extracting images from video files

In [1]:
import os
import cv2

In [2]:
def preprocess_image(img, img_size=128):
    # Convert to grayscale
    # Note: we convert from BGR as VideoCapture 
    # converts the images to BGR color frame by defualt
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Make image square by filling with black padding
    h, w = img.shape
    img = cv2.copyMakeBorder(
        img,
        top=0,
        right=max(0, h-w),
        bottom=max(0, w-h),
        left=0,
        borderType=0,
    )

    # Resize image to specified size
    img = cv2.resize(img, (img_size, img_size))
    return img

In [3]:
def extract_images_from_video(video, target_dir=None, file_prefix=None, img_size=128, silent=False):    
    """
    Function that saves every single frame of a video as images.
    Converts to greyscale, pads with black to make square images and resizes.
    """
    
    vidcap = cv2.VideoCapture(video)
    count = 0

    # Where to save images
    if not target_dir:
        target_dir = os.getcwd() + "/images_from_video"
        os.makedirs(target_dir, exist_ok=True) 

    # Use video file name as prefix if not specified
    if not file_prefix:
        file_prefix = video.rpartition(os.sep)[2].split(".")[0]

    success = True    
    while success:
        # Read image from video
        success, image = vidcap.read()
        
        if not success:
            continue
        
        # Convert to greyscale, make square and resize
        image = preprocess_image(image, img_size)

        # Save to file
        count += 1
        file_name = file_prefix + "_" + str(count) + ".jpg"
        path = os.path.join(target_dir, file_name)
        cv2.imwrite(path, image)

        # Check that image is not corrupted
        if cv2.imread(path) is None:
            if not silent:
                print(f"WARNING: image corrupted at path {path}")
            os.remove(path)
        else:
            if not silent:
                print(f'Image successfully written at {path}')
